# Idea is to create groups manually using NLP

In [1]:
import numpy as np
import pandas as pd

import gensim

# import string

In [2]:
df = pd.read_excel("Online_Retail_edited.xlsx")

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
items = df['Description'].value_counts().reset_index()
items.rename(columns = {'index':'Item_Name', 'Description':'Count'}, inplace=True)
items

,Item_Name,Count
0,WHITE HANGING HEART T-LIGHT HOLDER,2369
1,REGENCY CAKESTAND 3 TIER,2200
2,JUMBO BAG RED RETROSPOT,2159
3,PARTY BUNTING,1727
4,LUNCH BAG RED RETROSPOT,1638
...,...,...
4219,SET 36 COLOURING PENCILS DOILEY,1
4220,I LOVE LONDON MINI RUCKSACK,1
4221,sold as 22467,1
4222,SET 10 CARDS 12 DAYS OF XMAS 17059,1


In [4]:
len(items)

4224

### Only the required column is chosen, next is to use pretrained model and define groups

Possible groups:
    - FOOD
    - FURNITURE
    - STATIONERY
    - KITCHEN
    - ELECTRONICS
    - CLOTHES
    - CHEMICALS

In [5]:
groups = ['food','furniture','stationery','kitchen','electronics','clothes','chemicals']


In [6]:
model_name = r'/Users/subhagr/Desktop/My files/GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_name, binary=True)

In [7]:
vocab_list = list(model.key_to_index.keys())

In [8]:
def similarity(w1,w2):
    if((w1 in vocab_list) and (w2 in vocab_list)):
        return model.similarity(w1,w2)
    else: return 0

def similar_max(sentence):
    words = sentence.split()
    simmap = {'GROUPS':groups}
    max_sim = ('NONE',0)
    for each_word in words:
        temp = []
        for name in groups:
            simm = similarity(each_word.lower(),name)
            temp.append(simm)
            if(simm>max_sim[1]):
                max_sim = (name, simm)
            
        simmap[each_word] = temp
    
    return pd.DataFrame(simmap),max_sim

In [9]:
model.similarity('FOOD', "CUTLERY",)

KeyError: "Key 'CUTLERY' not present"

In [10]:
similarity('FOOD', "CUTLERY")

0

### The similarity function simply removes the error due to the absence of a word from W2V 

In [11]:
items['Item_Name'][0].split()


['WHITE', 'HANGING', 'HEART', 'T-LIGHT', 'HOLDER']

In [12]:
result = similar_max(items.iloc[1,0])
print(result[0])
print(result[1])

        GROUPS   REGENCY  CAKESTAND         3      TIER
0         food  0.079215          0  0.066109  0.025720
1    furniture  0.153682          0  0.033308  0.032501
2   stationery  0.188328          0  0.019708 -0.013772
3      kitchen  0.098494          0 -0.011125 -0.012686
4  electronics  0.001788          0  0.072788  0.032658
5      clothes  0.092584          0  0.069405 -0.034275
6    chemicals  0.003785          0  0.016075 -0.003606
('stationery', 0.18832812)


In [13]:
items.head()

,Item_Name,Count
0,WHITE HANGING HEART T-LIGHT HOLDER,2369
1,REGENCY CAKESTAND 3 TIER,2200
2,JUMBO BAG RED RETROSPOT,2159
3,PARTY BUNTING,1727
4,LUNCH BAG RED RETROSPOT,1638


In [14]:
items['Category'] = items['Item_Name'].apply(lambda x: similar_max(str(x))[1][0])

In [15]:
items

,Item_Name,Count,Category
0,WHITE HANGING HEART T-LIGHT HOLDER,2369,clothes
1,REGENCY CAKESTAND 3 TIER,2200,stationery
2,JUMBO BAG RED RETROSPOT,2159,clothes
3,PARTY BUNTING,1727,stationery
4,LUNCH BAG RED RETROSPOT,1638,food
...,...,...,...
4219,SET 36 COLOURING PENCILS DOILEY,1,stationery
4220,I LOVE LONDON MINI RUCKSACK,1,clothes
4221,sold as 22467,1,furniture
4222,SET 10 CARDS 12 DAYS OF XMAS 17059,1,stationery


In [16]:
# items.to_csv("Category_list.csv")

In [17]:
categories = {}
for i in range(len(items)):
    categories[items['Item_Name'][i]] = items['Category'][i]

In [18]:
df['Category'] = df['Description'].apply(lambda x: categories[x])

In [19]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,UnitPrceImp,Category
0,536365.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2.55,clothes
1,536365.0,71053.0,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,3.39,electronics
2,536365.0,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2.75,clothes
3,536365.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,3.39,food
4,536365.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,3.39,clothes
...,...,...,...,...,...,...,...,...,...,...
541904,581587.0,22613.0,PACK OF 20 SPACEBOY NAPKINS,12.0,2011-12-09 12:50:00,0.85,12680.0,France,0.85,stationery
541905,581587.0,22899.0,CHILDREN'S APRON DOLLY GIRL,6.0,2011-12-09 12:50:00,2.10,12680.0,France,2.1,kitchen
541906,581587.0,23254.0,CHILDRENS CUTLERY DOLLY GIRL,4.0,2011-12-09 12:50:00,4.15,12680.0,France,4.15,furniture
541907,581587.0,23255.0,CHILDRENS CUTLERY CIRCUS PARADE,4.0,2011-12-09 12:50:00,4.15,12680.0,France,4.15,furniture


In [20]:
df['Category'].value_counts()

kitchen        133236
furniture      108832
clothes        103306
stationery      81637
food            68472
electronics     27635
chemicals       18445
NONE              346
Name: Category, dtype: int64

In [21]:
df.drop('UnitPrice', axis=1, inplace=True)

In [22]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country,UnitPrceImp,Category
0,536365.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,17850.0,United Kingdom,2.55,clothes
1,536365.0,71053.0,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,17850.0,United Kingdom,3.39,electronics
2,536365.0,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,17850.0,United Kingdom,2.75,clothes
3,536365.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,17850.0,United Kingdom,3.39,food
4,536365.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,17850.0,United Kingdom,3.39,clothes
...,...,...,...,...,...,...,...,...,...
541904,581587.0,22613.0,PACK OF 20 SPACEBOY NAPKINS,12.0,2011-12-09 12:50:00,12680.0,France,0.85,stationery
541905,581587.0,22899.0,CHILDREN'S APRON DOLLY GIRL,6.0,2011-12-09 12:50:00,12680.0,France,2.1,kitchen
541906,581587.0,23254.0,CHILDRENS CUTLERY DOLLY GIRL,4.0,2011-12-09 12:50:00,12680.0,France,4.15,furniture
541907,581587.0,23255.0,CHILDRENS CUTLERY CIRCUS PARADE,4.0,2011-12-09 12:50:00,12680.0,France,4.15,furniture


In [23]:
small = df[df['StockCode'].apply(lambda x : True if len(str(x))<=4 else False)]

In [24]:
small

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country,UnitPrceImp,Category
45,536370.0,POST,POSTAGE,3.0,2010-12-01 08:45:00,12583.0,France,18,stationery
141,C536379,D,Discount,-1.0,2010-12-01 09:41:00,14527.0,United Kingdom,27.5,electronics
386,536403.0,POST,POSTAGE,1.0,2010-12-01 11:27:00,12791.0,Netherlands,15,stationery
1123,536527.0,POST,POSTAGE,1.0,2010-12-01 13:04:00,12662.0,Germany,18,stationery
1423,536540.0,C2,CARRIAGE,1.0,2010-12-01 14:05:00,14911.0,EIRE,50,kitchen
...,...,...,...,...,...,...,...,...,...
541540,581498.0,DOT,DOTCOM POSTAGE,1.0,2011-12-09 10:26:00,1.0,United Kingdom,1714.17,stationery
541541,C581499,M,Manual,-1.0,2011-12-09 10:28:00,15498.0,United Kingdom,224.69,kitchen
541730,581570.0,POST,POSTAGE,1.0,2011-12-09 11:59:00,12662.0,Germany,18,stationery
541767,581574.0,POST,POSTAGE,2.0,2011-12-09 12:09:00,12526.0,Germany,18,stationery


In [25]:
small_list = list(small['StockCode'].value_counts().index)

In [26]:
small_list+= ['BANK CHARGES', 'AMAZONFEE','DCGS0068']


In [27]:
small_list

['POST',
 'DOT',
 'M',
 'C2',
 'D',
 'S',
 'CRUK',
 'PADS',
 'B',
 'm',
 'BANK CHARGES',
 'AMAZONFEE',
 'DCGS0068']

In [37]:
def replace_cat(stc, cat):
    if((stc in small_list)or ('gift' in str(stc))): return 'NONE'
    else: return cat

In [38]:
df['Category'] = df.apply(lambda x: replace_cat(x['StockCode'], x["Category"]), axis=1)

In [39]:
df[df['Category']=='NONE']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country,UnitPrceImp,Category
45,536370.0,POST,POSTAGE,3.0,2010-12-01 08:45:00,12583.0,France,18.00,NONE
141,C536379,D,Discount,-1.0,2010-12-01 09:41:00,14527.0,United Kingdom,27.50,NONE
386,536403.0,POST,POSTAGE,1.0,2010-12-01 11:27:00,12791.0,Netherlands,15.00,NONE
1123,536527.0,POST,POSTAGE,1.0,2010-12-01 13:04:00,12662.0,Germany,18.00,NONE
1423,536540.0,C2,CARRIAGE,1.0,2010-12-01 14:05:00,14911.0,EIRE,50.00,NONE
...,...,...,...,...,...,...,...,...,...
541540,581498.0,DOT,DOTCOM POSTAGE,1.0,2011-12-09 10:26:00,1.0,United Kingdom,1714.17,NONE
541541,C581499,M,Manual,-1.0,2011-12-09 10:28:00,15498.0,United Kingdom,224.69,NONE
541730,581570.0,POST,POSTAGE,1.0,2011-12-09 11:59:00,12662.0,Germany,18.00,NONE
541767,581574.0,POST,POSTAGE,2.0,2011-12-09 12:09:00,12526.0,Germany,18.00,NONE


In [35]:
df['UnitPrceImp'] = df['UnitPrceImp'].apply(lambda x: 0 if isinstance(x,str) else x)

In [40]:
df[df['StockCode'].apply(lambda x: True if('gift' in str(x)) else False)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country,UnitPrceImp,Category
38248,539492.0,gift_0001_40,Dotcomgiftshop Gift Voucher £40.00,1.0,2010-12-20 10:14:00,1.0,United Kingdom,34.04,NONE
42057,539958.0,gift_0001_50,Dotcomgiftshop Gift Voucher £50.00,1.0,2010-12-23 13:26:00,1.0,United Kingdom,42.55,NONE
44725,540238.0,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1.0,2011-01-05 14:44:00,1.0,United Kingdom,25.53,NONE
44794,540238.0,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1.0,2011-01-05 14:44:00,1.0,United Kingdom,17.02,NONE
55589,540995.0,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1.0,2011-01-13 09:30:00,1.0,United Kingdom,16.67,NONE
91278,544089.0,gift_0001_20,Dotcomgiftshop Gift Voucher £20.00,1.0,2011-02-15 17:51:00,1.0,United Kingdom,16.67,NONE
94054,544323.0,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1.0,2011-02-17 15:51:00,1.0,United Kingdom,25.00,NONE
95034,544434.0,gift_0001_30,Dotcomgiftshop Gift Voucher £30.00,1.0,2011-02-18 16:12:00,1.0,United Kingdom,25.00,NONE
112442,545895.0,gift_0001_10,Dotcomgiftshop Gift Voucher £10.00,1.0,2011-03-07 17:14:00,1.0,United Kingdom,8.33,NONE
145463,548893.0,gift_0001_40,Dotcomgiftshop Gift Voucher £40.00,1.0,2011-04-04 15:54:00,1.0,United Kingdom,33.33,NONE


In [41]:
# df.to_csv('Retail_grouped.csv')